In [9]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("train.csv")
#print(train_data["Name"][:10])
test_data = pd.read_csv("test.csv")
test_target = pd.read_csv('gender_submission.csv')

def get_title(data):
    title = []
    for name in data['Name']:
        #title.append(name.split()[1][:-1])
        title.append(name.split(',')[1].split('.')[0].strip())
    return title

#title = get_title()
#print(title[:20])
train_data['Title'] = get_title(train_data)
test_data['Title'] = get_title(test_data)
train_data.loc[766, 'Title']='Mr'
test_data.loc[293, 'Title']='Mr'
test_data.loc[88, 'Title']='Mrs'
test_data.loc[152, 'Fare']=round(test_data[(test_data['Age']>55)&(test_data['Title']=='Mr')]['Fare'].mean())
y_train = np.array(train_data["Survived"])
y_test = np.array(test_target['Survived'])
train_data.drop('Survived', axis=1, inplace=True)
p_ID = test_data["PassengerId"]
display(train_data.tail())
display(test_data.tail())

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,Rev
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,Miss
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,Miss
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,Mr
890,891,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,Mr


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,Mr
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Dona
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,Mr
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,Master


In [10]:
#t = train_data[train_data.Title=="Miss"]
#print(t)
#print("Avg age of 'Miss' title", train_data[train_data.Title=="Miss"]['Age'].mean())
#print("Avg age of 'Miss' title travelling without parents",
#       round(train_data[(train_data.Title=="Miss") & (train_data.Parch==0)]['Age'].mean()))
#print("Avg age of 'Miss' title travelling with parents",
#      round(train_data[(train_data.Title=='Miss')&(train_data.Parch!=0)]['Age'].mean()))

avg_girl_age = round(train_data[(train_data.Title=='Miss')&(train_data.Parch!=0)]['Age'].mean())
avg_lady_age = round(train_data[(train_data.Title=="Miss") & (train_data.Parch==0)]['Age'].mean())
avg_boy_age = round(train_data[train_data.Title=='Master']['Age'].mean())
avg_men_age = round(train_data[train_data.Title=='Mr']['Age'].mean())
avg_women_age = round(train_data[train_data.Title=='Mrs']['Age'].mean())
print(avg_girl_age, avg_lady_age, avg_boy_age, avg_men_age, avg_women_age)

12.0 28.0 5.0 32.0 36.0


In [11]:
def fix_age(data):
    age_nan = data[np.isnan(data['Age'])].index
    #print(age_nan)
    for i in age_nan:
        #print(i)
        person = data.iloc[i]
        if person.Title == 'Master':
            data.loc[i, 'Age'] = avg_boy_age
            #retur avg_boy_age
        elif person.Title == 'Mr':
            data.loc[i, 'Age'] = avg_men_age
        elif person.Title=='Miss' and person.Parch==0:
            data.loc[i, 'Age'] = avg_lady_age
        elif person.Title=='Miss' and person.Parch!=0:
            data.loc[i, 'Age'] = avg_girl_age
        elif person.Title=='Mrs':
            data.loc[i, 'Age'] = avg_women_age
    return data

In [12]:
train_data = fix_age(train_data)
test_data = fix_age(test_data)
#print(test_data.iloc[201])
#print(test_data.iloc[354])
train_data['Age']=train_data['Age'].apply(lambda x:np.log(x+1))
train_data['Fare']=train_data['Fare'].apply(lambda x:np.log(x+1))
test_data['Age']=test_data['Age'].apply(lambda x:np.log(x+1))
test_data['Age']=test_data['Age'].apply(lambda x:np.log(x+1))
#train_data[(train_data.Title=='Master')&(np.isnan(train_data.Age))]['Age'] = avg_boy_age
#print(train_data[np.isnan(train_data['Age'])])
#display(train_data.head(n=10))
#display(test_data.head())
#print(test_data.iloc[201])
#print(test_data.iloc[354])

In [13]:
train_data.drop(labels=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Title'], axis=1, inplace=True)
test_data.drop(labels=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Title'], axis=1, inplace=True)
X_train = np.array(pd.get_dummies(train_data))
X_test = pd.get_dummies(test_data)
X_train[:5]
X_test[:5]

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1.519411,0,0,7.8292,0,1,0,1,0
1,3,1.583341,1,0,7.0000,1,0,0,0,1
2,2,1.637663,0,0,9.6875,0,1,0,1,0
3,3,1.466077,0,0,8.6625,0,1,0,0,1
4,3,1.419607,1,1,12.2875,1,0,0,0,1


In [14]:
#print(X_train[np.isnan(X_train['Embarked_Q'])])
print(X_test[np.isnan(X_test['Age'])])

Empty DataFrame
Columns: [Pclass, Age, SibSp, Parch, Fare, Sex_female, Sex_male, Embarked_C, Embarked_Q, Embarked_S]
Index: []


In [15]:
from sklearn.tree import DecisionTreeClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC

#clf = LogisticRegression(max_iter=1000, random_state=0)
clf = DecisionTreeClassifier(max_depth=15, random_state=0)
#clf = SVC(random_state=0)
#clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

print("Acccuracy on training set: {}".format(clf.score(X_train, y_train)))
print("Accuracy on test set: {}".format(clf.score(X_test, y_test)))

Acccuracy on training set: 0.9674523007856342
Accuracy on test set: 0.9066985645933014


In [22]:
predictions = clf.predict(X_test)
df = pd.DataFrame({'PassengerId':p_ID, 'Survived':predictions})
df.to_csv(r'C:\Users\USER\Desktop\my_pythonfiles_\kaggle\titanic_survival\\Solution.csv', index=False)